In [22]:
from os import listdir, walk
from os.path import isfile, join
import cv2
import os
import io
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
import base64
import numpy as np
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import Markdown, display
import ntpath
from requests import post, put

## Test our dogs on our API

In [ ]:
API_KEY = "[YourSecretKeyCanBeAnything]"   # This is a secret key on your app service - only requests with this key will be allowed
WEB_APP_URL = "https://[your app service name].azurewebsites.net"

def stringToRGB(base64_string):
    imgdata = base64.b64decode(str(base64_string))
    image = Image.open(io.BytesIO(imgdata))
    return cv2.cvtColor(np.array(image), cv2.IMREAD_ANYCOLOR)


def predict(dog):

    try:
        headers = {
            # Request headers
            "Content-Type": "application/json",
            "Ocp-Apim-Subscription-Key": API_KEY,
        }

        body = {
        "values": [
            {
                "recordId": "0",
                "data": {
                    "images": {
                        "data": rottweiler_test
                    }
                }
            }
        ]
    }

        url = f"{WEB_APP_URL}/api/extraction"

        resp = post(url=url, json=body, headers=headers)

        result_response = resp.json()
    
        return result_response

    except Exception as e:
        print('Exception', e)
        
    return result_response
        
# Download test data 
with open("../data/test/rottweiler.txt") as m:
    rottweiler_test = m.read()
  
result_response = predict(rottweiler_test)

assert(result_response['values'][0]['data']['amllabel'] == 'rottweiler')
print("You should see a rottweiler")
img = stringToRGB(rottweiler_test)
plt.imshow(img)



## Deploy the PowerSkill to Azure Search¶


### Create the data source

In [ ]:
# Let's create a data source
ACS_KEY = ""   # Your ACS API Key
ACS_URL = "https://[your search name].search.windows.net"   # Your ACS URL format https://[your ACS instance].search.windows.net
DATA_SOURCE = "aml-classifier-datasource"  # The name for your data source
CONTAINER_NAME = "dogs"   # The name of your blob container where the images are
BLOB_CONNECTION_STRING = ""   # This is your azure blob connection string

json_text = {
    "name" : DATA_SOURCE,
    "type" : "azureblob",
    "credentials" : { "connectionString" : BLOB_CONNECTION_STRING},    
    "container" : { "name" : CONTAINER_NAME }  # The name of the container where the data files are
}

headers = {
    "api-key": ACS_KEY,     
    "Content-Type": "application/json",

}

try:
    url = f"{ACS_URL}/datasources?api-version=2020-06-30"
    resp = post(url=url, json=json_text, headers=headers)

    result_response = resp.json()
    if resp.status_code == 403:
        print("Authorisation Failed: Check that your API KEY value is correct")
        
    if resp.status_code == 400:
        print(resp.text)
        
    if resp.status_code == 201:
        print("Success creating data source")
        
except Exception as e:
    print('Exception creating data source', e)

### Now we create the index

In [ ]:
INDEX_NAME = "aml-classifier-index"  # The name for the index

json_text = {
      "name" : INDEX_NAME,
      "fields": [
        { "name": "id", "type": "Edm.String", "key": True, "searchable": False },
        { "name": "url", "type": "Edm.String", "searchable": True },
        { "name": "file_name", "type": "Edm.String", "searchable": False },
        { "name": "size", "type": "Edm.Int64", "searchable": False },
        { "name": "last_modified", "type": "Edm.DateTimeOffset", "searchable": False },
        { "name": "content", "type": "Edm.String", "searchable": True, "filterable": False, "sortable": False, "facetable": False },
        { "name": "amllabel", "type": "Collection(Edm.String)", "searchable": True, "filterable": False, "sortable": False, "facetable": False },
        { "name": "images", "type": "Collection(Edm.String)", "searchable": True, "filterable": False, "sortable": False, "facetable": False },
        {
            "name": "categories",
            "type": "Collection(Edm.ComplexType)",
            "fields": [
                {
                    "name": "name",
                    "type": "Edm.String",
                    "searchable": True,
                    "filterable": False,
                    "facetable": False
                },
                {
                    "name": "score",
                    "type": "Edm.Double",
                    "searchable": False,
                    "filterable": False,
                    "facetable": False
                },
                {
                    "name": "detail",
                    "type": "Edm.ComplexType",
                    "fields": [
                        {
                            "name": "celebrities",
                            "type": "Collection(Edm.ComplexType)",
                            "fields": [
                                {
                                    "name": "name",
                                    "type": "Edm.String",
                                    "searchable": True,
                                    "filterable": False,
                                    "facetable": False
                                },
                                {
                                    "name": "faceBoundingBox",
                                    "type": "Collection(Edm.ComplexType)",
                                    "fields": [
                                        {
                                            "name": "x",
                                            "type": "Edm.Int32",
                                            "searchable": False,
                                            "filterable": False,
                                            "facetable": False
                                        },
                                        {
                                            "name": "y",
                                            "type": "Edm.Int32",
                                            "searchable": False,
                                            "filterable": False,
                                            "facetable": False
                                        }
                                    ]
                                },
                                {
                                    "name": "confidence",
                                    "type": "Edm.Double",
                                    "searchable": False,
                                    "filterable": False,
                                    "facetable": False
                                }
                            ]
                        },
                        {
                            "name": "landmarks",
                            "type": "Collection(Edm.ComplexType)",
                            "fields": [
                                {
                                    "name": "name",
                                    "type": "Edm.String",
                                    "searchable": True,
                                    "filterable": False,
                                    "facetable": False
                                },
                                {
                                    "name": "confidence",
                                    "type": "Edm.Double",
                                    "searchable": False,
                                    "filterable": False,
                                    "facetable": False
                                }
                            ]
                        }
                    ]
                }
            ]
        },
        {
            "name": "description",
            "type": "Collection(Edm.ComplexType)",
            "fields": [
                {
                    "name": "tags",
                    "type": "Collection(Edm.String)",
                    "searchable": True,
                    "filterable": False,
                    "facetable": False
                },
                {
                    "name": "captions",
                    "type": "Collection(Edm.ComplexType)",
                    "fields": [
                        {
                            "name": "text",
                            "type": "Edm.String",
                            "searchable": True,
                            "filterable": False,
                            "facetable": False
                        },
                        {
                            "name": "confidence",
                            "type": "Edm.Double",
                            "searchable": False,
                            "filterable": False,
                            "facetable": False
                        }
                    ]
                }
            ]
        },
        {
            "name": "faces",
            "type": "Collection(Edm.ComplexType)",
            "fields": [
                {
                    "name": "age",
                    "type": "Edm.Int32",
                    "searchable": False,
                    "filterable": False,
                    "facetable": False
                },
                {
                    "name": "gender",
                    "type": "Edm.String",
                    "searchable": False,
                    "filterable": False,
                    "facetable": False
                },
                {
                    "name": "faceBoundingBox",
                    "type": "Collection(Edm.ComplexType)",
                    "fields": [
                        {
                            "name": "x",
                            "type": "Edm.Int32",
                            "searchable": False,
                            "filterable": False,
                            "facetable": False
                        },
                        {
                            "name": "y",
                            "type": "Edm.Int32",
                            "searchable": False,
                            "filterable": False,
                            "facetable": False
                        }
                    ]
                }
            ]
        },
        {
            "name": "tags",
            "type": "Collection(Edm.ComplexType)",
            "fields": [
                {
                    "name": "name",
                    "type": "Edm.String",
                    "searchable": True,
                    "filterable": False,
                    "facetable": False
                },
                {
                    "name": "confidence",
                    "type": "Edm.Double",
                    "searchable": False,
                    "filterable": False,
                    "facetable": False
                }
            ]
        }
      ]
}

try:
    url = f"{ACS_URL}/indexes?api-version=2020-06-30"
    resp = post(url=url, json=json_text, headers=headers)
    
    result_response = resp.json()
    if resp.status_code == 403:
        print("Authorisation Failed: Check that your API KEY value is correct")
        
    if resp.status_code == 400:
        print(f"Error", resp.text)    
            
    if resp.status_code == 201:
        print("Success creating index")
        
except Exception as e:
    print('Exception creating index', e)

## Now we create the skill sets

In [ ]:
# Note we are passing in the secret header key and the inference API URL to the skillset
SKILLSET_NAME = "aml-classifier-skillset"  # The name of your skillset
COGSVC_KEY = ""  #  This is your Cognitive Services key that resides in the same region as ACS (used to compare custom vision captions and object detection)

json_text = {
    "description": "Crack documents.",
    "skills": [
        {
            "@odata.type": "#Microsoft.Skills.Vision.ImageAnalysisSkill",
            "context": "/document/normalized_images/*",
            "defaultLanguageCode": "en",
            "visualFeatures": [
                "tags",
                "categories",
                "description",
                "faces",
                "brands"
            ],
            "inputs": [
                {
                    "name": "image",
                    "source": "/document/normalized_images/*"
                }
            ],
            "outputs": [
                {
                    "name": "categories"
                },
                {
                    "name": "tags"
                },
                {
                    "name": "description"
                },
                {
                    "name": "faces"
                },
                {
                    "name": "brands"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Custom.WebApiSkill",
            "description": "A custom skill that runs inference on an image",
            "uri": f"{WEB_APP_URL}/api/extraction",
            "timeout": "PT160S",
            "batchSize": 1,
            "context": "/document/normalized_images/*",
            "httpHeaders": {
                "Ocp-Apim-Subscription-Key": API_KEY
            },
            "httpMethod": "POST",
            "inputs": [
            {
                "name": "images",
                    "source": "/document/normalized_images/*"
                }
            ],
            "outputs": [
                {
                    "name": "amllabel"
                }
            ]
        }
    ],
    "cognitiveServices": {
        "@odata.type": "#Microsoft.Azure.Search.CognitiveServicesByKey",
        "description": "cogsvc",
        "key": COGSVC_KEY
    }
}



try:
    url = f"{ACS_URL}/skillsets/{SKILLSET_NAME}?api-version=2020-06-30"
    resp = put(url=url, json=json_text, headers=headers)
    
    result_response = resp.json()
    if resp.status_code == 403:
        print("Authorisation Failed: Check that your API KEY value is correct")
    
    if resp.status_code == 400:
        print(f"Error", resp.text)

    if resp.status_code == 201:
        print("Success creating skillset")
        
except Exception as e:
    print('Exception creating skillset', e)


### Now we create the indexer

In [ ]:
json_text = {
    "name": "aml-classifier-indexer",
    "dataSourceName": DATA_SOURCE,
    "targetIndexName": INDEX_NAME,
    "skillsetName": SKILLSET_NAME,
    "parameters": {
        "configuration": {
            "allowSkillsetToReadFileData": True,
            "imageAction": "generateNormalizedImagePerPage"
        }
    },
    "outputFieldMappings": [
        {
            "sourceFieldName": "/document/normalized_images/*/amllabel",
            "targetFieldName": "amllabel"
        },
        {
            "sourceFieldName": "/document/normalized_images/*/categories/*",
            "targetFieldName": "categories"
        },
        {
            "sourceFieldName": "/document/normalized_images/*/tags/*",
            "targetFieldName": "tags"
        },
        {
            "sourceFieldName": "/document/normalized_images/*/description",
            "targetFieldName": "description"
        },
        {
            "sourceFieldName": "/document/normalized_images/*/faces/*",
            "targetFieldName": "faces"
        }
    ]
}


try:
    url = f"{ACS_URL}/indexers?api-version=2020-06-30"
    resp = post(url=url, json=json_text, headers=headers)
    
    result_response = resp.json()
    if resp.status_code == 403:
        print("Authorisation Failed: Check that your API KEY value is correct")

    if resp.status_code == 400:
        print(f"Error", resp.text)
        
    if resp.status_code == 201:
        print("Success creating indexer")
        
except Exception as e:
    print('Exception creating indexer', e)

### Let's go and test the ACS index 

In [ ]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__
# Create the BlobServiceClient object which will be used to create a container client
blob_service_client = BlobServiceClient.from_connection_string(BLOB_CONNECTION_STRING)

def printmd(string):
    display(Markdown(string))

search_term = "rottweiler"

# Create a client
credential = AzureKeyCredential(ACS_KEY)
client = SearchClient(endpoint=ACS_URL,
                      index_name=INDEX_NAME,
                      credential=credential)


results = client.search(search_text=search_term, top=20)

for result in results:
    try:   
        dog = str(base64.urlsafe_b64decode(result["id"]+ "=="))
        dog = str(dog[1:-3])
        file_name = ntpath.basename(dog)
        ind = file_name.find(".")
        file_name = file_name[:ind] + '.jpg'
        blob_client = blob_service_client.get_blob_client(container=CONTAINER_NAME, blob=file_name)
        download_file_path = os.path.join("", file_name)
        print(f"Downloading blob to {download_file_path}")

        with open(download_file_path, "wb") as download_file:
            download_file.write(blob_client.download_blob().readall())
    except Exception as InvalidBase64:
        print(InvalidBase64)
        continue
        
    image_data = open(download_file_path, "rb").read()
    image = Image.open(BytesIO(image_data))    
    plt.imshow(image)
    plt.axis("off")
    _ = plt.title("", size="x-large", y=-0.1)
    plt.show()
    
    printmd('**Custom Vision Tags**')
    printmd(' '.join(result['description'][0]['tags']))
    
    printmd('**Custom Vision Caption**')
    printmd(result['description'][0]['captions'][0]['text'])
    
    printmd('**AML Label**')
    printmd(result['amllabel'][0])
